Getting the following information from every T&C html page
1. Tokenized text from the main body (with src package)

In [ ]:
# uncomment if you need to install any of the packages below
# !python -m pip install --upgrade pip
# !pip install langid
# !pip install somajo
# !pip install selenium
# !pip install webdriver-manager
# !pip install -U selenium --user
# !pip install webdriver-manager
# !pip install readability_lxml

In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
from enum import Enum
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from src.StructuredLegalExtraction import extractTandC
import src.ContentExtractor.ContentExtractorTypes
from src.ContentExtractor import ContentExtractor
from src.ContentExtractor.ContentExtractor import getMainContent
from src.ContentExtractor.TreeUtilities import getFrequencyOfStyles
import requests
import readability_lxml
from readability_lxml import Document
import ast
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

In [2]:
# links_df = pd.read_excel('data/URLs_Final.xlsx
links_df = pd.read_excel('data/2-urls-all.xlsx')
links_df

,Unnamed: 0,Firm,URL,Industry
0,0,Whiteleys Financial Management LLP,http://www.whiteleysaccountants.co.uk/terms-an...,Investment Firms
1,1,Whiteleys Financial Management LLP,http://www.whiteleysaccountants.co.uk/privacy-...,Investment Firms
2,2,Whiteleys Financial Management LLP,http://www.whiteleysaccountants.co.uk/news-ite...,Investment Firms
3,4,Pioneer Point Partners LLP,https://pioneerpoint.com/privacy-and-cookie-po...,Investment Firms
4,5,Engage Wealth Management Limited,https://engagewm.co.uk/privacy-notice/,Investment Firms
...,...,...,...,...
4823,4837,Heddington Insurance (U.K.) Limited,https://www.chevron.com/privacy,Insurers
4824,4838,Heddington Insurance (U.K.) Limited,https://www.chevron.com/terms-of-use,Insurers
4825,4839,Lady Grover's Hospital Fund for Officers' Fami...,https://forceshealth.co.uk/terms-and-conditions/,Insurers
4826,4840,Lady Grover's Hospital Fund for Officers' Fami...,https://forceshealth.co.uk/wordpress/wp-conten...,Insurers


In [ ]:
# uncomment and change index for running code in batches
# links = links_df.iloc[10:15]['URL']
# firms = links_df.iloc[10:15]['Firm']

links = links_df['URL']
firms = links_df['Firm']

In [ ]:
%%time 
driver = webdriver.Chrome(ChromeDriverManager().install())

def extractTandD_multiple(links, contentExtractor=src.ContentExtractor.ContentExtractorTypes.ContentExtractor.NaiveStyleAndShortTextExclusion,
                          threshold=0.85, driver=None):
    close = False
    if driver is None:
        close = True
        driver = webdriver.Chrome(executable_path='../chromedriver')
    results = []
    links_count = 0
    for link in links:
        try:
            results.append(extractTandC(link, contentExtractor, threshold, driver))
        except:
            results.append(np.NaN)
        links_count += 1
        clear_output(wait=True)
        print(f'Progress: {links_count/len(links)*100}%')
    if close:
        driver.close()
    return results

token_text = extractTandD_multiple(links, driver = driver)

In [ ]:
# recursive function to clean token_text 
def sentence_function(l):
    if len(l) == 0: 
        pass
    elif (len(l) >= 1) & (type(l[0]) == str):
        pure_sentences.append(l)
    elif (len(l) >= 1) & (type(l[0]) == dict):
        for i in np.arange(len(l)):
            sentence_function(l[i]['text'])
    else:
        for i in np.arange(len(l)):
            sentence_function(l[i])
            
pure_sentences_list = []

for i in texts:
    pure_sentences = []
    sentence_function(i)
    pure_sentences_list.append(pure_sentences)
    
pure_sentencesss = []
for i in pure_sentences_list:
    pure_sentencess = []
    for j in i:
        try:
            full_sentence = " ".join(j)
            pure_sentencess.append(full_sentence)
        except: 
            pass
    pure_sentencesss.append(pure_sentencess)

In [ ]:
# Export
links_df['Text'] = pure_sentencesss
links_df.to_csv("data/3-text-extraction.csv", mode='a', header=False)